# Optimized ways to Read Large CSVs in Python

### Author: Shachi Kaul
### Motive: For handling large CSVs, problems such as crashing of kernel and Out of Memory error.

# Milestones
- Import Libraries
- Prepare huge-sized dataframe
- How quick your import option is?
    - Size of data to be imported
    - pandas.read_csv()
    - pandas.read_sv(chunksize)
    - dask.dataframe.read_csv()

In [1]:
import pandas as pd
import numpy as np
from dask import dataframe as dd
import time
import os
from dask.distributed import Client

D:\Users\SKaul7\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


## Prepare a huge-sized dataframe

In [5]:
df = pd.DataFrame(data=np.random.randint(99999, 99999999, size=(10000000,14)),columns=['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14'])

#### Add a string column to dataframe with a random string

In [6]:
df['C15'] = pd.util.testing.rands_array(5, 10000000)

#### Data Analysis

In [7]:
df.shape

(10000000, 15)

In [8]:
df.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15
0,52254190,36823947,66293076,60079342,79253435,68729334,92031556,67294877,74581862,14922253,26790682,57105776,16926222,32063311,P4VVc
1,49911927,84844430,5308032,59251686,50766178,39805376,14923736,80428468,63539315,6949030,92130220,37094061,29512227,28492994,ub4dX
2,42268205,24499377,98750370,36486031,61128869,63010166,27666012,8269366,14794948,16546148,43646474,78714163,24961500,10464317,Sha70
3,44611348,64750253,42787155,23769160,54813654,46183635,54433623,12208968,54123662,54669284,97926233,29959834,87839414,3913639,oCfLW
4,68515682,18827316,97732951,57636534,86741144,69492670,3046228,6164679,82089039,3507701,80976752,76799542,3139903,10830273,mObAD


In [9]:
# Store data in CSV format
df.to_csv("huge_data.csv")

## How quick your import option is?
- pandas.read_csv()
- pandas.read_csv(chunksize)
- dask.dataframe.read_csv()

In [16]:
# Just in case, to clean up RAM
import gc
gc.collect()

0

#### Size of data to be imported

In [14]:
print(os.path.getsize('huge_data.csv')/1024/1024/1024 , "GB")

1.298035160638392 GB


##### Pandas.read_csv()

In [10]:
start = time.time()
df = pd.read_csv('huge_data.csv')
end = time.time()
print("Read csv without chunks: ",(end-start),"sec")

Read csv without chunks:  26.88872528076172 sec


In [11]:
%%time
df.iloc[:,1].mean()

Wall time: 41 ms


50041578.5786115

##### pandas.read_csv(chunksize)

In [2]:
start = time.time()
chunk = pd.read_csv('huge_data.csv',chunksize=1000000)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")
pd_df = pd.concat(chunk)

Read csv with chunks:  0.013001203536987305 sec


In [3]:
type(chunk)

pandas.io.parsers.TextFileReader

In [4]:
type(pd_df)

pandas.core.frame.DataFrame

In [6]:
%%time
pd_df.iloc[:,1].mean()

Wall time: 16 ms


50041578.5786115

##### dask.dataframe.read_csv()

#### Start Dask Client for Dashboard
It will provide a dashboard which is useful to gain insight on the computation.

In [2]:
client = Client(processes=False)
client

Client Scheduler: inproc://10.220.104.115/15460/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 2 Memory: 8.59 GB


In [2]:
start = time.time()
dask_df = dd.read_csv('huge_data.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.052001237869262695 sec


In [9]:
%%time
dask_df.iloc[:,1].mean().compute()

Wall time: 19.9 s


50041578.5786115

## Conclusion
- Dask performed much faster while reading large CSV prepared.
- Even pandas can perform better while loading up in chunks via chunksize argument.